In [1]:
from spinbox import *
from itertools import count
seeder = count(0,1) # generates a list of counting numbers for rng "seeds" (not really seeds in Numpy, but indices of seeds)

def npr(name, thing):
    print(name,"\n",thing,"\n"+16*"-")

# Tutorial 3: Basic imaginary-time propagation in the full basis space

This notebook will show how to approach imaginary-time propagation of `HilbertState` objects using a simple interaction.

Note that `spinbox` does not have any built-in kinetic operator stuff. We are focused on the 2- and 3-body interactions, although any operator can be made using a `HilbertOperator`.

Consider the following imaginary-time propagator amplitude between states $S$ and $S'$ of a 2-particle system with spin and isospin.

$$\langle S' | \exp \left[  - \frac{\delta\tau}{2} A^\sigma_{1x2x} \sigma_{1x} \sigma_{2x} \right] | S \rangle $$

First, instantiate some random states. Here I have a 2-body system with spin-isospin variables.

In [2]:
s = HilbertState(2, ketwise=True, isospin=True).random(next(seeder))
sp = HilbertState(2, ketwise=False, isospin=True).random(next(seeder))
print(s,"\n",sp)

HilbertState ket of 2 particles:
[[ 0.02733963-0.11834735j]
 [-0.02872577-0.06877844j]
 [ 0.13925783+0.08950772j]
 [ 0.02281019+0.22669115j]
 [-0.11647957-0.02794945j]
 [ 0.07862767+0.29713305j]
 [ 0.28355058-0.14464442j]
 [ 0.20593968+0.07643472j]
 [-0.15302494+0.19645666j]
 [-0.27516179+0.02044267j]
 [-0.13552901-0.1616715j ]
 [ 0.0089862 -0.20042619j]
 [-0.5055704 -0.099531j  ]
 [-0.04757554+0.04788072j]
 [-0.27091929-0.2195382j ]
 [-0.15922916-0.04548455j]] 
 HilbertState bra of 2 particles:
[[ 0.06726313+0.00773135j  0.15991648-0.05692262j  0.06431496-0.15218755j -0.25364133-0.05005887j  0.17621486+0.00158476j  0.08688057-0.05364225j -0.10451044+0.25187142j  0.11310651+0.19594481j
   0.07095892-0.52768986j  0.05724878-0.36767001j  0.00553199-0.03401694j  0.10641004-0.08217346j -0.14334049+0.04158262j -0.03170814+0.04229867j -0.09383778+0.41220769j  0.11655704-0.21643929j]]


The coupling $A^\sigma_{1x2x}$ is some real number determined by other calculations, so we will just plug in some number. For more complicated situations, we should use a `TwoBodyCoupling` for this.

We can compute the amplitude "exactly" using the Pade approximant exponentiation, like so.

In [3]:
coupling = 3.14
dt = 0.01
force = HilbertOperator(2).apply_sigma(0,0).apply_sigma(1,0).scale(coupling)  # note that isospin is True by default
propagator = force.scale(- 0.5j * dt).exp()
amplitude_exact = sp * propagator * s
print("exact = ", amplitude_exact)

exact =  [[0.26982964+0.16993701j]]


Next, lets compute this again using a Hubbard-Stratonovich transform, using the `HilbertPropagatorHS` class. 

The HS and RBM propagator classes all have built-in methods for the usual forces, as well as methods `onebody()` and `twobody_sample()` which apply one-body propagators and samples of the "two-body" summand for the appropriate method.

Note that here we are "balancing" the distribution: for each sample of the Gaussian auxiliary field $x$ we also evaluate at $-x$.

In [4]:
n_samples = 10000
prop_hs = HilbertPropagatorHS(2, dt)
gaussian_fields = np.random.standard_normal(size=n_samples)
operator_i = HilbertOperator(2).apply_sigma(0,0)
operator_j = HilbertOperator(2).apply_sigma(1,0)
amplitude_distribution_hs = np.zeros(2*n_samples, dtype=complex)
for ix,x in enumerate(gaussian_fields):
    amplitude_distribution_hs[2*ix:2*ix+1]= sp * prop_hs.twobody_sample(coupling, x, operator_i, operator_j) * s
    x_flip = -x
    amplitude_distribution_hs[2*ix+1:2*ix+2]= sp * prop_hs.twobody_sample(coupling, x_flip, operator_i, operator_j) * s
amplitude_hs = np.mean(amplitude_distribution_hs)
amplitude_hs_unc = np.std(amplitude_distribution_hs)/np.sqrt(n_samples)
print(f"HS = {amplitude_hs} +/- {amplitude_hs_unc}")
print(f"relative error = {(amplitude_exact - amplitude_hs)/amplitude_exact}")

HS = (0.26885909666394164+0.16357751156822775j) +/- 0.0005535387565291832
relative error = [[0.01320325+0.01525324j]]


That's in agreement with the exact. Finally, let's use `HilbertPropagatorRBM` to do the same. We also balance this calculation: for each sample of the binary auxiliary field $h$ we also evaluate at $1-h$.

In [5]:
prop_rbm = HilbertPropagatorRBM(2, dt)
binary_fields = np.random.randint(0,2,size=n_samples)
amplitude_distribution_rbm = np.zeros(2*n_samples, dtype=complex)
for ih,h in enumerate(binary_fields):
    amplitude_distribution_rbm[2*ih:2*ih+1] = sp * prop_rbm.twobody_sample(coupling, h, operator_i, operator_j) * s
    h_flip = 1-h
    amplitude_distribution_rbm[2*ih+1:2*ih+2] = sp * prop_rbm.twobody_sample(coupling, h_flip, operator_i, operator_j) * s
amplitude_rbm = np.mean(amplitude_distribution_rbm)
amplitude_rbm_unc = np.std(amplitude_distribution_rbm)/np.sqrt(n_samples)
print(f"RBM = {amplitude_rbm} +/- {amplitude_rbm_unc}")
print(f"relative error = {(amplitude_exact - amplitude_rbm)/amplitude_exact}")

RBM = (0.2688824741229454+0.16367333610635662j) +/- 0.00024147255792024894
relative error = [[0.01298108+0.01503804j]]


For a single term of the interaction, the RBM is exact. Later we will do a calculation with a sum of non-commuting interaction terms, which introduces a finite error.

While we are here, let's try a three-body force too. We can do the same thing but compute this 3-body propagator amplitude for a 3-particle system. We do not have a Hubbard-Stratonovich transformation for 3-body forces, but we can use the RBM.

$$\langle S' | \exp \left[  - \frac{\delta\tau}{2} A^\sigma_{1x2x3x} \sigma_{1x} \sigma_{2x} \sigma_{3x} \right] | S \rangle $$

Note that the scaling for three-body is worse, so depending on `n_samples` this may take a while. Later, we will look at how to do general calculations using the `Integrator` class, which has thread-parallelization.

In [6]:
s = HilbertState(3, ketwise=True, isospin=True).random(next(seeder))
sp = HilbertState(3, ketwise=False, isospin=True).random(next(seeder))

coupling = 3.14
dt = 0.001
force = HilbertOperator(3).apply_sigma(0,0).apply_sigma(1,0).apply_sigma(2,0).scale(coupling) 
propagator = force.scale(- 0.5j * dt).exp()
amplitude_exact = sp * propagator * s
print("exact = ", amplitude_exact)

n_samples = 1000
prop_rbm = HilbertPropagatorRBM(3, dt)
operator_i = HilbertOperator(3).apply_sigma(0,0)
operator_j = HilbertOperator(3).apply_sigma(1,0)
operator_k = HilbertOperator(3).apply_sigma(2,0)
binary_fields = np.random.randint(0,2,size=(n_samples,4))
amplitude_distribution_rbm = np.zeros(2*n_samples, dtype=complex)
for ih,h in enumerate(tqdm(binary_fields)):
    amplitude_distribution_rbm[2*ih:2*ih+1] = sp * prop_rbm.threebody_sample(coupling, h, operator_i, operator_j, operator_k) * s
    amplitude_distribution_rbm[2*ih+1:2*ih+2] = sp * prop_rbm.threebody_sample(coupling, 1-h, operator_i, operator_j, operator_k) * s
amplitude_rbm = np.mean(amplitude_distribution_rbm)
amplitude_rbm_unc = np.std(amplitude_distribution_rbm)/np.sqrt(n_samples)
print(f"RBM = {amplitude_rbm} +/- {amplitude_rbm_unc}")
print(f"error = {(amplitude_exact - amplitude_rbm)}")
print(f"relative error = {(amplitude_exact - amplitude_rbm)/amplitude_exact}")

exact =  [[0.12757498-0.02596224j]]


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:05<00:00, 196.80it/s]

RBM = (0.12790383966416727-0.026331264488369286j) +/- 0.003442323925234451
error = [[-0.00032886+0.00036903j]]
relative error = [[-0.0030405+0.00227389j]]
